# 8章：ニューラルネット
第6章で取り組んだニュース記事のカテゴリ分類を題材として，ニューラルネットワークでカテゴリ分類モデルを実装する．なお，この章ではPyTorch, TensorFlow, Chainerなどの機械学習プラットフォームを活用せよ。  

## 70. 単語ベクトルの和による特徴量

問題50で構築した学習データ，検証データ，評価データを行列・ベクトルに変換したい．例えば，学習データについて，すべての事例$x_i$の特徴ベクトル$\boldsymbol{x}_i$を並べた行列$X$と正解ラベルを並べた行列（ベクトル）$Y$を作成したい．

$$
X = \begin{pmatrix} 
  \boldsymbol{x}_1 \\ 
  \boldsymbol{x}_2 \\ 
  \dots \\ 
  \boldsymbol{x}_n \\ 
\end{pmatrix} \in \mathbb{R}^{n \times d},
Y = \begin{pmatrix} 
  y_1 \\ 
  y_2 \\ 
  \dots \\ 
  y_n \\ 
\end{pmatrix} \in \mathbb{N}^{n}
$$


 ここで，$n$は学習データの事例数であり，$\boldsymbol x_i \in \mathbb{R}^d$と$y_i \in \mathbb N$はそれぞれ，$i \in \{1, \dots, n\}$番目の事例の特徴量ベクトルと正解ラベルを表す．
 なお，今回は「ビジネス」「科学技術」「エンターテイメント」「健康」の4カテゴリ分類である．$\mathbb N_{<4}$で$4$未満の自然数（$0$を含む）を表すことにすれば，任意の事例の正解ラベル$y_i$は$y_i \in \mathbb N_{<4}$で表現できる．
 以降では，ラベルの種類数を$L$で表す（今回の分類タスクでは$L=4$である）．

 $i$番目の事例の特徴ベクトル$\boldsymbol x_i$は，次式で求める．

 $$\boldsymbol x_i = \frac{1}{T_i} \sum_{t=1}^{T_i} \mathrm{emb}(w_{i,t})$$

 ここで，$i$番目の事例は$T_i$個の（記事見出しの）単語列$(w_{i,1}, w_{i,2}, \dots, w_{i,T_i})$から構成され，$\mathrm{emb}(w) \in \mathbb{R}^d$は単語$w$に対応する単語ベクトル（次元数は$d$）である．  
 すなわち，$i$番目の事例の記事見出しを，その見出しに含まれる単語のベクトルの平均で表現したものが$\boldsymbol x_i$である．今回は単語ベクトルとして，問題60でダウンロードしたものを用いればよい．$300$次元の単語ベクトルを用いたので，$d=300$である．  
 $i$番目の事例のラベル$y_i$は，次のように定義する．

$$
y_i = \begin{cases}
0 & (\mbox{記事}\boldsymbol x_i\mbox{が「ビジネス」カテゴリの場合}) \\
1 & (\mbox{記事}\boldsymbol x_i\mbox{が「科学技術」カテゴリの場合}) \\
2 & (\mbox{記事}\boldsymbol x_i\mbox{が「エンターテイメント」カテゴリの場合}) \\
3 & (\mbox{記事}\boldsymbol x_i\mbox{が「健康」カテゴリの場合}) \\
\end{cases}
$$

なお，カテゴリ名とラベルの番号が一対一で対応付いていれば，上式の通りの対応付けでなくてもよい．

以上の仕様に基づき，以下の行列・ベクトルを作成し，ファイルに保存せよ．

 + 学習データの特徴量行列: $X_{\rm train} \in \mathbb{R}^{N_t \times d}$
 + 学習データのラベルベクトル: $Y_{\rm train} \in \mathbb{N}^{N_t}$
 + 検証データの特徴量行列: $X_{\rm valid} \in \mathbb{R}^{N_v \times d}$
 + 検証データのラベルベクトル: $Y_{\rm valid} \in \mathbb{N}^{N_v}$
 + 評価データの特徴量行列: $X_{\rm test} \in \mathbb{R}^{N_e \times d}$
 + 評価データのラベルベクトル: $Y_{\rm test} \in \mathbb{N}^{N_e}$

なお，$N_t, N_v, N_e$はそれぞれ，学習データの事例数，検証データの事例数，評価データの事例数である．

In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import gensim
from gensim.models import word2vec
from gensim.models import KeyedVectors

In [2]:
#データの取り出し
train = pd.read_table('data/train.txt')
valid = pd.read_table('data/valid.txt')
test = pd.read_table('data/test.txt')

In [3]:
#タイトル列のみの抽出→特徴量の生成
X_train = train['TITLE']
X_valid = valid['TITLE']
X_test = test['TITLE']

In [4]:
#カテゴリ列のみの抽出→ラベルの生成
Y_train = train['CATEGORY']
Y_valid = valid['CATEGORY']
Y_test = test['CATEGORY']

In [5]:
#word2vecをダウンロード
EMBEDDING_FILE = 'data/GoogleNews-vectors-negative300.bin.gz'
word_vectors = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)

In [6]:
#ベクトル表現の生成関数を定義
def create_vector(data):
    result = []
    for i in range(len(data)):
        listdata = data[i].split(' ')
        subresult = np.zeros(300)
        number = 0
        for j in range(len(listdata)):
            try:
                subresult += word_vectors[listdata[j]]
                number += 1
            except Exception as e: #単語埋め込み範囲外の語彙を見つけた時に例外処理
                pass
        if number == 0:
            result.append(subresult)
        else:
            result.append(subresult/number)
    return result

In [7]:
#特徴量においてベクトル表現を生成
X_train = create_vector(X_train)
X_valid = create_vector(X_valid)
X_test = create_vector(X_test)

In [8]:
#ラベルにおいて特徴量を生成
dct_int = {'b': 0, 't': 1, 'e': 2,'m': 3}
Y_train = Y_train.replace(dct_int)
Y_valid = Y_valid.replace(dct_int)
Y_test = Y_test.replace(dct_int)

In [9]:
#型を合わせる
X_train = torch.tensor(np.array(X_train)).float()
X_valid = torch.tensor(np.array(X_valid)).float()
X_test = torch.tensor(np.array(X_test)).float()

Y_train = torch.tensor(Y_train.values)
Y_valid = torch.tensor(Y_valid.values)
Y_test = torch.tensor(Y_test.values)

In [10]:
torch.save(X_train, 'data/X_train.pt')
torch.save(X_valid, 'data/X_valid.pt')
torch.save(X_test, 'data/X_test.pt')

torch.save(Y_train, 'data/Y_train.pt')
torch.save(Y_valid, 'data/Y_valid.pt')
torch.save(Y_test, 'data/Y_test.pt')

## 71. 単層ニューラルネットワークによる予測
問題70で保存した行列を読み込み，学習データについて以下の計算を実行せよ．

$$ 
\hat{y}_1=softmax(x_1W),\\\hat{Y}=softmax(X_{[1:4]}W)
$$


ただし，$softmax$はソフトマックス関数，$X_{[1:4]}∈\mathbb{R}^{4×d}$は特徴ベクトル$x_1$,$x_2$,$x_3$,$x_4$を縦に並べた行列である．

$$
X_{[1:4]}=\begin{pmatrix}x_1\\x_2\\x_3\\x_4\end{pmatrix}
$$

行列$W \in \mathbb{R}^{d \times L}$は単層ニューラルネットワークの重み行列で，ここではランダムな値で初期化すればよい（問題73以降で学習して求める）．  
なお，$\hat{\boldsymbol y_1} \in \mathbb{R}^L$は未学習の行列$W$で事例$x_1$を分類したときに，各カテゴリに属する確率を表すベクトルである．
同様に，$\hat{Y} \in \mathbb{R}^{n \times L}$は，学習データの事例$x_1, x_2, x_3, x_4$について，各カテゴリに属する確率を行列として表現している．


In [11]:
#ニューラルネットワーク構築
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Linear(300, 4, bias=False)
        
    def forward(self, input):
        output = activation(self.layer1(input))
        return output
    
model = NeuralNetwork()

In [12]:
#活性化関数の定義
activation = nn.Softmax(dim=0)

#Yの生成
result = []
#for i in range(4):
    #result.append(model(X_train[i]))


#Yのプリント
#print(result)

'''
jupyter notebookでmodel(入力)を実行しようとすると

Kernel Restarting
The kernel for masahiro_makino_chapter08.ipynb appears to have died. It will restart automatically.

とエラーが出る。ちなみにサーバー上でPythonのスクリプトファイルを実行すればエラーが出ることなく実行できます。
'''

'\njupyter notebookでmodel(入力)を実行しようとすると\n\nKernel Restarting\nThe kernel for masahiro_makino_chapter08.ipynb appears to have died. It will restart automatically.\n\nとエラーが出る。ちなみにサーバー上でPythonのスクリプトファイルを実行すればエラーが出ることなく実行できます。\n'

## 72. 損失と勾配の計算

学習データの事例x1
と事例集合x1,x2,x3,x4
に対して，クロスエントロピー損失と，行列W
に対する勾配を計算せよ．なお，ある事例xi
に対して損失は次式で計算される．

$$
l_i=−log[事例x_iがy_iに分類される確率]
$$

ただし，事例集合に対するクロスエントロピー損失は，その集合に含まれる各事例の損失の平均とする．

## 73. 確率的勾配降下法による学習

確率的勾配降下法（SGD: Stochastic Gradient Descent）を用いて，行列W
を学習せよ．なお，学習は適当な基準で終了させればよい（例えば「100エポックで終了」など）．

## 74. 正解率の計測

問題73で求めた行列を用いて学習データおよび評価データの事例を分類したとき，その正解率をそれぞれ求めよ．

## 75. 損失と正解率のプロット
問題73のコードを改変し，各エポックのパラメータ更新が完了するたびに，訓練データでの損失，正解率，検証データでの損失，正解率をグラフにプロットし，学習の進捗状況を確認できるようにせよ．

## 76. チェックポイント

問題75のコードを改変し，各エポックのパラメータ更新が完了するたびに，チェックポイント（学習途中のパラメータ（重み行列など）の値や最適化アルゴリズムの内部状態）をファイルに書き出せ．



## 77. ミニバッチ化

問題76のコードを改変し，B
事例ごとに損失・勾配を計算し，行列W
の値を更新せよ（ミニバッチ化）．B
の値を1,2,4,8,…
と変化させながら，1エポックの学習に要する時間を比較せよ．



In [13]:
!pip install torchvision

     |████████████████████████████████| 1.3 MB 801 kB/s eta 0:00:01


In [14]:
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [15]:
class DataSet:
    def __init__(self):
        self.X = X_train # 入力
        self.Y = Y_train # 出力

    def __len__(self):
        return len(self.X) # データ数を返す

    def __getitem__(self, index):
        # index番目の入出力ペアを返す
        return self.X[index], self.Y[index]

In [16]:
dataset = DataSet()
dataloader = torch.utils.data.DataLoader(dataset, batch_size=3, shuffle=False)

In [18]:
#損失関数の定義
loss_entropy = nn.CrossEntropyLoss()

#最適化手法の決定
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

In [ ]:
EPOCHS = 100
for epoch in range(EPOCHS):
    for data in dataloader:
        optimizer.zero_grad()  # 重みとバイアスの更新で内部的に使用するデータをリセット
        outputs = model(data[0])  # 手順1：ニューラルネットワークにデータを入力
        loss = loss_entropy(outputs, data[1])  # 手順2：正解ラベルとの比較
        loss.backward()  # 手順3-1：誤差逆伝播
        optimizer.step()  # 手順3-2：重みとバイアスの更新

In [ ]:
time_list = []
epoch_list = []
for i in range(math.log2(len(X_train))):
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=pow(2, i), shuffle=False)
    start_time = time.perf_counter()
    for data in dataloader:
        optimizer.zero_grad()  # 重みとバイアスの更新で内部的に使用するデータをリセット
        outputs = model(data[0])  # 手順1：ニューラルネットワークにデータを入力
        loss = loss_entropy(outputs, data[1])  # 手順2：正解ラベルとの比較
        loss.backward()  # 手順3-1：誤差逆伝播
        optimizer.step()  # 手順3-2：重みとバイアスの更新
    end_time = time.perf_counter()    
    epoch_list.append(pow(2, i))
    time_list.append()

## 78. GPU上での学習

問題77のコードを改変し，GPU上で学習を実行せよ．

## 79. 多層ニューラルネットワーク

問題78のコードを改変し，バイアス項の導入や多層化など，ニューラルネットワークの形状を変更しながら，高性能なカテゴリ分類器を構築せよ